### Code Hist.

 dev-shryu  
 
 - CODE  
    &ensp; : Initialization - 코드 환경설정 및 Device(GPU), Module 설정
 - DATE  
    &ensp; 2023-10-18 Created  
 - DESC  
    &ensp; : Code 공통화 - 코드 초기설정(Env, Device, Module)
 - DATA  
    &ensp;&ensp; <"Input">  
    1) None  
    &ensp;&ensp;- Period :   
    &ensp;&ensp;- Interval : 

    &ensp;&ensp; <"Output">  
    1) None  
    &ensp;&ensp;- Period :   
    &ensp;&ensp;- Interval : 
 - Link  
    1) ipynb 작성  
    &nbsp; :  https://stackoverflow.com/questions/47061626/how-to-get-tab-space-in-markdown-cell-of-jupyter-notebook

## 00. Environment  
- Module Install / Device / GPU

#### 00-01. Module Install (PIP)

In [1]:
## ▶ For Environment
## ▶ [필요시에만] Anaconda 필수 Module 
# !conda update conda
# !conda update --all

# !python -m pip install --upgrade pip

## ▶ 개발환경 Freeze
# !pip freeze

In [2]:
## ▶ For Dev
## ▶ (필요시 설치 가능) Protobuf (관련 오류가 날 경우에 사용했음)
## https://velog.io/@alvinlee9/ImportError-cannot-import-name-builder-from-google.protobuf.internal
# !pip install --upgrade protobuf
# !pip install protobuf==3.19.4
# from google.protobuf.internal import builder as _builder
# from google.protobuf import descriptor as _descriptor
# from google.protobuf import descriptor_pool as _descriptor_pool

## ▶ Python 기본 Module
# !pip install numpy

## ▶ (필요시 설치 가능) Python Optional Module
# !pip install --upgrade category_encoders
# !pip install gdown
# !pip install kaggle
# !pip install lightgbm

## ▶ (필요시 설치 가능) For TCN 
# !pip uninstall tensorflow
# !pip install tensorflow==2.9.1
# !pip install tensorflow-gpu==2.10.0
## TCN 관련 Module
# !pip install tqdm
# !pip install feature-engine
# !pip install tensorflow-addons
## KerasTCN 라이브러리 (미사용)
# !pip install keras-tcn --user
# !pip install keras-tcn --no-dependencies  # without the dependencies if you already have TF/Numpy.
# !pip install tcn 



In [3]:
## Excel To Pandas DF
# !pip install openpyxl
# !pip install openpyxl --upgrade
# !pip install xlrd

In [4]:
##
# !pip install selenium
# !pip install seaborn
# !pip install webdriver_manager

#### 00-02-01. Device Check (GPU)

In [5]:
!nvidia-smi

Thu Oct 26 14:06:04 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.84                 Driver Version: 545.84       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0  On |                  Off |
| 44%   32C    P8              17W / 450W |   1159MiB / 24564MiB |     10%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(tf.__version__)

print(device_lib.list_local_devices())
print(tf.config.list_physical_devices('GPU'))
tf.test.is_gpu_available()

print(device_lib.list_local_devices())

2.10.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2966954799193082262
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22395486208
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13977388828844299925
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16317309197956204344
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22395486208
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10356707956405164035
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


#### 00-02-02. Device Config (GPU 사용 설정)

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

tf.debugging.set_log_device_placement(True)

In [8]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

c = tf.matmul(a, b)
print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


## 01. Code

### 01-01. Init

#### 01-01-01. Init_Module Import

In [10]:
## Module Import
import os
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import warnings
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series

import math, random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [11]:
## Init.
pd.options.display.float_format = '{:.10f}'.format

In [12]:
## Import_Torch
# import torch
# import torch.nn as nn
# from torch.nn.utils import weight_norm

## Import_Tensorflow
import tensorflow as tf
import tensorflow_addons as tfa

#### 01-01-02. Init_Config (Directory, Params)

In [13]:
# Define data root directory
str_dirData = "./KIER_EnergyUsage_2023-10-09/"
print(os.listdir(str_dirData))

# str_fileData = "KIER_ELEC_METER_HST_2023-10-04"

['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'ENV_SENSOR_HST_202309251452.csv', 'F1_MAIN_POWER_HST_202309251503.csv', 'F1_SUB_POWER_HST_202309251510.csv', 'HEATPUMP_HST_202309251520.csv', 'KIER_ALL_561_18F.csv', 'KIER_ALL_561_1F.csv', 'KIER_ALL_562_18F.csv', 'KIER_ALL_562_1F.csv', 'KIER_ALL_563_18F.csv', 'KIER_ALL_563_1F.csv', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_ELEC_561_18F.csv', 'KIER_ELEC_561_1F.csv', 'KIER_ELEC_562_18F.csv', 'KIER_ELEC_562_1F.csv', 'KIER_ELEC_563_18F.csv', 'KIER_ELEC_563_1F.csv', 'KIER_ELEC_METER_HST_2023-10-04.csv', 'KIER_GAS_METER_HST_2023-10-04.csv', 'KIER_HEAT_561_18F.csv', 'KIER_HEAT_561_1F.csv', 'KIER_HEAT_562_18F.csv', 'KIER_HEAT_562_1F.csv', 'KIER_HEAT_563_18F.csv', 'KIER_HEAT_563_1F.csv', 'KIER_HEAT_METER_HST_2023-10-04.csv', 'KIER_HOT_561_18F.csv', 'KIER_HOT_561_1F.csv', 'KIER_HOT_562_18F.csv', 'KIER_HOT_562_1F.csv', 'KIER_HOT_563_18F.csv', 'KIER_HOT_563_1F.c

In [14]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

### 01-02. Data Load (df_raw)

#### 01-02-01. KIER Data (Energy Usage)

In [15]:
## Define data root directory
str_dirData = "./KIER_EnergyUsage_2023-10-09/"
str_ex = ".csv"

## 동/층/호 입력
## 동 : 561 / 562 / 563
int_dong = 561
## 층 : 1 / 18
int_floor = 1
int_ho = ""

## Elec Data
str_fileData_Elec = str("KIER_ELEC_" + str(int_dong) + "_" + str(int_floor) + "F")
## Heat Data
str_fileData_Heat = str("KIER_HEAT_" + str(int_dong) + "_" + str(int_floor) + "F")
## Hot Data
str_fileData_Hot = str("KIER_HOT_" + str(int_dong) + "_" + str(int_floor) + "F")
## Water Data
str_fileData_Water = str("KIER_WATER_" + str(int_dong) + "_" + str(int_floor) + "F")

df_Energy_Elec = pd.read_csv(str_dirData + str_fileData_Elec + str_ex)
df_Energy_Heat = pd.read_csv(str_dirData + str_fileData_Heat + str_ex)
df_Energy_Hot = pd.read_csv(str_dirData + str_fileData_Hot + str_ex)
df_Energy_Water = pd.read_csv(str_dirData + str_fileData_Water + str_ex)

In [16]:
## 임시 Code (다음에 각각 Src에서 제거할 것)
## 아래 1분단위 Code를 수행하기 위함
df_Energy_Elec = df_Energy_Elec.drop(columns=['ELEC_METER_DATE'], axis = 1)
df_Energy_Heat = df_Energy_Heat.drop(columns=['HEAT_METER_DATE'], axis = 1)
df_Energy_Hot = df_Energy_Hot.drop(columns=['HOT_METER_DATE'], axis = 1)
df_Energy_Water = df_Energy_Water.drop(columns=['WATER_METER_DATE'], axis = 1)

In [17]:
## 임시 Code (다음에 각각 Src에서 컬럼명 변경부여할 것)
df_Energy_Elec = df_Energy_Elec.rename(columns={'ELEC_ACTUAL_ACCU_EFF' : str('ELEC_ACCU_' + str(int_dong) + '_' + str(int_floor) + 'F')})
df_Energy_Heat = df_Energy_Heat.rename(columns={'HEAT_ACCU_HEAT' : str('HEAT_ACCU_' + str(int_dong) + '_' + str(int_floor) + 'F')})
df_Energy_Hot = df_Energy_Hot.rename(columns={'HOT_ACCU_HEAT' : str('HOT_ACCU_' + str(int_dong) + '_' + str(int_floor) + 'F')})
df_Energy_Water = df_Energy_Water.rename(columns={'WATER_ACCU_FLOW' : str('WATER_ACCU_' + str(int_dong) + '_' + str(int_floor) + 'F')})

In [18]:
print(df_Energy_Elec.shape)
print(df_Energy_Elec.columns)
print(df_Energy_Heat.shape)
print(df_Energy_Heat.columns)
print(df_Energy_Hot.shape)
print(df_Energy_Hot.columns)
print(df_Energy_Water.shape)
print(df_Energy_Water.columns)

(57413, 11)
Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'code_day_of_the_week',
       'day_of_the_week', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO',
       'ELEC_ACCU_561_1F', 'MINUTE'],
      dtype='object')
(42544, 12)
Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'code_day_of_the_week',
       'day_of_the_week', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO',
       'HEAT_ACCU_561_1F', 'MINUTE', 'HEAT_INST_ACTUAL_CALC'],
      dtype='object')
(57383, 11)
Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'code_day_of_the_week',
       'day_of_the_week', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO',
       'HOT_ACCU_561_1F', 'MINUTE'],
      dtype='object')
(57732, 11)
Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'code_day_of_the_week',
       'day_of_the_week', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO',
       'WATER_ACCU_561_1F', 'MINUTE'],
      dtype='object')


In [20]:
# print(df_Energy_Elec.head(5))
# print(df_Energy_Heat.head(5))
# print(df_Energy_Hot.head(5))
# print(df_Energy_Water.head(5))

   YEAR  MONTH  DAY  HOUR  code_day_of_the_week day_of_the_week  \
0  2022      7   17    23                     6          Sunday   
1  2022      7   17    23                     6          Sunday   
2  2022      7   17    23                     6          Sunday   
3  2022      7   17    23                     6          Sunday   
4  2022      7   18     0                     0          Monday   

   HOUSE_ID_DONG  HOUSE_ID_HO_PRE  \
0            561                1   
1            561                1   
2            561                1   
3            561                1   
4            561                1   

                                    HOUSE_ID_HO  WATER_ACCU_561_1F  MINUTE  
0  NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=      51.8681000000      20  
1  NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=      51.8681000000      30  
2  NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=      51.8681000000      40  
3  NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=      51.9031000000